<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/GitHub_Copilot_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GitHub Copilot 실습 노트북
본 노트북은 VS Code 등 IDE에서 **GitHub Copilot**을 사용해 코드를 생성/완성하는 연습을 하면서,
Colab에서 결과를 **실행·검증**할 수 있도록 구성되었습니다.

**구성**
1) 함수 구현(문서화 주도 개발)
2) 테스트 코드 자동 생성(간단 assert로 대체 검증)
3) 데이터 처리(pandas)
4) 리팩토링 & 주석/Docstring 개선
5) 작은 API 스켈레톤 설계(코드 스니펫 실행)
6) 프롬프트 패턴 & 모범 사례 정리

> ⚠️ Copilot 자체는 Colab에 직접 통합되지 않습니다. **IDE에서 Copilot을 써서 코드를 만든 뒤**, 이 노트북 셀에 붙여 넣고 실행하세요.

In [ ]:
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35


## Lab 1 — Docstring으로 함수 구현하기
아래 TODO는 **사양(명세)**입니다. IDE에서 Copilot에 다음과 같이 힌트를 주세요:

**Copilot 힌트(주석으로 입력):**
```
문자열 리스트에서 알파뉴메릭이 아닌 문자를 제거하고, 소문자로 통일한 뒤, 길이가 0인 항목은 제거한다.
중복은 제거하되 입력 순서는 유지한다.
예: ['Hello!', 'world', 'HELLO', '  ', 'A&B'] -> ['hello', 'world', 'ab']
엣지케이스: None, 숫자, 리스트가 아닌 항목이 섞여 들어오면 무시한다.
시간복잡도: O(n)
```
완성한 함수를 아래 셀에 붙여 넣고 실행하세요.

In [ ]:
# TODO: Copilot으로 작성한 함수를 아래에 붙여 넣으세요.
# 함수명 예시: normalize_unique(strings)

def normalize_unique(strings):
    import re
    if not isinstance(strings, (list, tuple)):
        return []
    seen, out = set(), []
    for x in strings:
        if not isinstance(x, str):
            continue
        s = re.sub(r'[^0-9a-zA-Z]', '', x).lower().strip()
        if not s:
            continue
        if s not in seen:
            seen.add(s); out.append(s)
    return out

# 간단 검증
tests = [
    (['Hello!', 'world', 'HELLO', '  ', 'A&B'], ['hello','world','ab']),
    (['a','A','a!!','b','B_','',None,123], ['a','b']),
    ('not-a-list', []),
]
for inp, exp in tests:
    got = normalize_unique(inp)
    print(inp, '->', got, '✓' if got==exp else f'✗ expected {exp}')


['Hello!', 'world', 'HELLO', '  ', 'A&B'] -> ['hello', 'world', 'ab'] ✓
['a', 'A', 'a!!', 'b', 'B_', '', None, 123] -> ['a', 'b'] ✓
not-a-list -> [] ✓


In [ ]:
import re
from typing import List, Any

def clean_alphanum_list(items: List[Any]) -> List[str]:
    seen = set()
    result = []
    for item in items:
        if not isinstance(item, str):
            continue
        # Remove non-alphanumeric, make lowercase
        cleaned = re.sub(r'[^a-zA-Z0-9]', '', item).lower()
        if cleaned and cleaned not in seen:
            seen.add(cleaned)
            result.append(cleaned)
    return result

# Example usage:
print(clean_alphanum_list(['Hello!', 'world', 'HELLO', '  ', 'A&B', None, 123, ['nested']]))
# Output: ['hello', 'world', 'ab']

['hello', 'world', 'ab']


## Lab 2 — 테스트 코드 자동 생성(Copilot 제안 활용)
Copilot에게 아래와 같이 요청해 **경계 값/에러 케이스**를 포함한 테스트 시나리오를 제안받으세요.

**Copilot 힌트:**
```
normalize_unique 함수에 대한 테스트 케이스 10개를 작성해줘.
엣지 케이스(빈 리스트, None 포함, 특수문자만, 매우 긴 입력 등)도 포함해줘.
각 케이스는 (input, expected) 형태의 튜플로 정의하고, 반복문으로 검증해줘.
```

In [ ]:
# Copilot이 제안한 테스트 케이스를 아래 리스트에 추가/수정하세요.
more_tests = [
    ([], []),
    (['!!!','***'], []),
    (['A','a','A','b','B','c'], ['a','b','c']),
    (['  mixed-CASE  ', '\tTabs\t', '\nNew\n'], ['mixedcase','tabs','new']),
    ([None, 'valid', None], ['valid']),
    (['abc123', 'abc-123', 'ABC_123'], ['abc123']),
    (['   ', 'x', 'x  ', ' x '], ['x']),
    (['안녕', '세상'], []),
    (['a']*10000 + ['b'], ['a','b']),
    (['  spaced  text  '], ['spacedtext']),
]
ok = True
for inp, exp in more_tests:
    got = normalize_unique(inp)
    if got != exp:
        print('FAIL:', inp, '->', got, 'expected', exp)
        ok = False
print('All good!' if ok else 'Some tests failed.')


All good!


In [ ]:
import re

def normalize_unique(items):
    seen = set()
    result = []
    for item in items:
        if not isinstance(item, str):
            continue
        cleaned = re.sub(r'[^a-zA-Z0-9]', '', item).lower()
        if cleaned and cleaned not in seen:
            seen.add(cleaned)
            result.append(cleaned)
    return result

# 테스트 케이스 작성
test_cases = [
    # 1. 기본 동작
    (['Hello!', 'world', 'HELLO', '  ', 'A&B'], ['hello', 'world', 'ab']),
    # 2. 빈 리스트
    ([], []),
    # 3. None 포함
    (['abc', None, 'ABC', None], ['abc']),
    # 4. 숫자 포함
    (['123', 456, '123', None], ['123']),
    # 5. 특수문자만
    (['!@#$%^', '   ', ''], []),
    # 6. 중복+대소문자
    (['Test', 'test', 'TEST', 'TeSt'], ['test']),
    # 7. 리스트가 아닌 항목
    (['foo', ['bar'], 'Bar', {'baz': 1}], ['foo', 'bar']),
    # 8. 매우 긴 입력
    (['a'*1000, 'A'*1000, 'b'*1000], ['a'*1000, 'b'*1000]),
    # 9. 알파벳+숫자 혼합
    (['A1b2C3!', 'a1B2c3', '123abc'], ['a1b2c3', '123abc']),
    # 10. 공백/탭/개행 포함
    (['   spaced out   ', '\tTabbed\t', '\nNewLine\n', ''], ['spacedout', 'tabbed', 'newline']),
]

# 테스트 실행
for i, (input_data, expected) in enumerate(test_cases, 1):
    output = normalize_unique(input_data)
    assert output == expected, f"Test case {i} failed: input={input_data}, expected={expected}, got={output}"
print("All test cases passed!")

All test cases passed!


## Lab 3 — 데이터 처리(pandas) 보일러플레이트 생성
다음 요구사항을 Copilot에 설명하고, pandas 코드를 생성해보세요.

**요구사항:**
1) 아래 샘플 CSV를 생성한다.
2) 결측치(NaN)는 제거한다.
3) `category`별 `value`의 합계와 평균을 구한다.
4) 합계 기준 상위 2개 카테고리만 남긴다.
5) 결과를 `result.csv`로 저장한다.

**Copilot 힌트:**
```
pandas로 위 요구사항을 수행하는 코드를 작성해줘. 코드에는 주석을 자세히 달아줘.
```

In [ ]:
import pandas as pd
import numpy as np

# 샘플 CSV 생성
df = pd.DataFrame({
    'category': ['A','B','A','C','B','A','C','B',None,'A'],
    'value': [10, 5, np.nan, 7, 3, 12, 1, 9, 11, 4]
})
df.to_csv('sample.csv', index=False)
pd.read_csv('sample.csv').head()


,category,value
0,A,10.0
1,B,5.0
2,A,NaN
3,C,7.0
4,B,3.0


In [ ]:
# TODO: Copilot이 생성한 pandas 처리 코드를 여기에 붙여넣고 실행하세요.
import pandas as pd
df = pd.read_csv('sample.csv')
df = df.dropna(subset=['category','value'])
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df.dropna(subset=['value'])
agg = df.groupby('category')['value'].agg(['sum','mean']).reset_index()
top2 = agg.sort_values('sum', ascending=False).head(2)
top2.to_csv('result.csv', index=False)
top2


,category,sum,mean
0,A,26.0,8.666667
1,B,17.0,5.666667


## Lab 4 — 리팩토링 & Docstring 개선
아래 **비효율적/가독성 낮은 코드**를 Copilot으로 리팩토링하세요.
- 목표: 가독성 향상, 예외 처리 추가, 타입 힌트/Docstring 보강, 작은 함수로 분리

**Copilot 힌트:**
```
다음 코드를 PEP8 스타일에 맞게 리팩토링하고, 타입 힌트와 풍부한 Docstring을 추가해줘.
가능하면 작은 함수로 분리하고, 예외 처리를 추가해줘.
```

In [ ]:
# BEFORE
def weird(a,b,c):
    r=[]
    for i in a:
        try:
            if i not in r:
                if isinstance(i,int) or isinstance(i,float):
                    if i>0:
                        r.append(i+b-c)
                else:
                    k = str(i).strip().lower()
                    if k and k not in r:
                        r.append(k)
        except Exception as e:
            pass
    return r

weird([1,2,2,3,' A ',None, -5, 0.0, 1.0], 10, 3)


[8, 9, 9, 10, 'a', 'none', 8.0]

In [ ]:
# AFTER (예시 정답 — 여러분의 결과와 달라도 무방합니다)
from typing import Iterable, List, Union, Any

Number = Union[int, float]

def _normalize_text(x: Any) -> str:
    return str(x).strip().lower()

def weird_refactored(a: Iterable[Any], b: Number, c: Number) -> List[Union[Number, str]]:
    """
    Post-process mixed values:
    - Positive numbers -> add (b - c)
    - Non-numbers -> normalized string (strip+lower)
    - Deduplicate while preserving order
    """
    out, seen = [], set()
    for x in a:
        try:
            if isinstance(x, (int, float)):
                if x > 0:
                    val = x + (b - c)
                    if val not in seen:
                        seen.add(val); out.append(val)
            else:
                s = _normalize_text(x)
                if s and s not in seen:
                    seen.add(s); out.append(s)
        except Exception:
            continue
    return out

weird_refactored([1,2,2,3,' A ',None, -5, 0.0, 1.0], 10, 3)


[8, 9, 10, 'a', 'none']

## Lab 5 — 작은 API 스켈레톤 만들기(FastAPI)
Colab에서 서버 실행은 제한적이므로, **설계/코드 검토** 중심으로 진행합니다.
Copilot에 다음을 요청하여 **FastAPI** 스니펫을 생성하세요.

**Copilot 힌트:**
```
FastAPI로 /health, /normalize 엔드포인트를 갖는 최소 API를 작성해줘.
/normalize는 Lab1의 normalize_unique 기능을 호출하고 JSON을 반환해줘.
uvicorn으로 실행하는 if __name__ == '__main__' 블록도 포함해줘.
```

In [ ]:
# 예시 스니펫 — 생성된 코드와 비교해보세요
from typing import List
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Items(BaseModel):
    data: List[str]

@app.get('/health')
def health():
    return {'status':'ok'}

@app.post('/normalize')
def normalize(items: Items):
    return {'result': normalize_unique(items.data)}

print('스키마와 핸들러 정의 예시입니다. (서버 실행은 생략)')


## Lab 6 — Copilot 프롬프트 패턴 & 모범 사례
**좋은 힌트 작성법**
- **의도/제약/입출력/엣지케이스**를 명시
- **예시** 제공 (입력→출력)
- **형식** 지정(함수명, 반환타입, 시간복잡도, 코딩 규칙 등)
- **컨텍스트** 유지(같은 파일/함수에서 연속 작성)

**나쁜 힌트의 예**
- "대충 만들어줘" (맥락·제약 부재)
- 입력/출력 예시 없음

**실전 템플릿**
```
역할: (예) 데이터 처리 유틸리티 함수 작성
목표: 입력 리스트를 정규화하여 중복 제거, O(n)
입력: List[str|Any]
출력: List[str]
규칙: 알파뉴메릭만, 소문자, 공백 제거, 비문자/None 무시, 순서 보존
예시: ['Hello!', 'world', 'HELLO', '  ', 'A&B'] -> ['hello','world','ab']
```